In [4]:
import stable_baselines3
from raiSimulationEnvATKDEF import RobotSimEnv
import robotic as ry
import numpy as np
import time

import os
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from stable_baselines3 import PPO
import gymnasium as gym

class RewardLoggerCallback(BaseCallback):
    def __init__(self, log_file: str, verbose: int = 0):
        super(RewardLoggerCallback, self).__init__(verbose)
        self.log_file = log_file
        self.episode_rewards = []
        self.current_episode_reward = 0

        # Create the log file if it doesn't exist
        if not os.path.exists(self.log_file):
            with open(self.log_file, 'w') as f:
                f.write("Episode,Total Reward\n")

    def _on_step(self) -> bool:
        # Check if the episode has ended by using `done`
        dones = self.locals["dones"]
        rewards = self.locals["rewards"]

        # Accumulate rewards for the current episode
        self.current_episode_reward += rewards[0]

        # If the episode is done, log the reward
        if dones[0]:
            self.episode_rewards.append(self.current_episode_reward)
            with open(self.log_file, 'a') as f:
                f.write(f"{len(self.episode_rewards)},{self.current_episode_reward}\n")
            # Reset the reward counter for the next episode
            self.current_episode_reward = 0

        return True

    def _on_training_end(self) -> None:
        # Optionally summarize results at the end of training
        print("Training finished. Total episodes:", len(self.episode_rewards))
        print("Episode rewards:", self.episode_rewards)

class CustomCheckpointCallback(BaseCallback):
    def __init__(self, save_freq, save_path, verbose=0):
        super(CustomCheckpointCallback, self).__init__(verbose)
        self.save_freq = save_freq
        self.save_path = save_path

    def _on_step(self) -> bool:
        # Save the model every `save_freq` steps
        if self.n_calls % self.save_freq == 0:
            model_path = f"{self.save_path}/model_checkpoint_{self.n_calls}_steps.zip"
            self.model.save(model_path)
            if self.verbose > 0:
                print(f"Model saved at step {self.n_calls} to {model_path}")
        return True

class StaticOpponentWrapper(gym.Wrapper):
    """
    A wrapper for a 1v1 environment where one agent is controlled by a fixed, pre-trained policy.
    """
    def __init__(self, env, static_policy_attacker, static_policy_defender,attacker=True,test=False):
        super(StaticOpponentWrapper, self).__init__(env)
        self.static_policy_attacker = static_policy_attacker
        self.static_policy_defender = static_policy_defender
        self.attacker = attacker
        self.test = test

    def step(self, action):
        # Get the static opponent's action
        obs = self.env.state  # Modify this based on how your env works
        attacker_action, _ = self.static_policy_attacker.predict(obs, deterministic=True)
        defender_action, _ = self.static_policy_defender.predict(obs, deterministic=True)
        # Combine both actions into a joint action
        if self.attacker:
            joint_action = (attacker_action, action)
        else:
            joint_action = (action,defender_action)
        
        if self.test:
            joint_action = (attacker_action, defender_action)

        # Step the environment with both actions
        obs, reward, done, truncated, info = self.env.step(joint_action)
        return obs, reward, done, truncated, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


In [ ]:

env = RobotSimEnv(render_mode='human')

policy_kwargs = dict(
    net_arch=[dict(pi=[256, 256])]  # 'pi' is the actor network, 'vf' is the critic network
)

model = PPO("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

reward_logger = RewardLoggerCallback(log_file="rewards_log_baranpath.csv")
model.learn(total_timesteps=100000,progress_bar=True,callback=reward_logger)

In [5]:
from stable_baselines3 import PPO
import time

env = RobotSimEnv(render_mode='human',staticAttacker=True, staticDefender=True)

attackModel = PPO.load("best_sword_model_attack")
defenceModel = PPO.load("shield_model_best")

wrapped_env = StaticOpponentWrapper(env, attackModel,defenceModel,attacker=True,test=True)

policy_kwargs = dict(
    net_arch=[dict(pi=[256, 256])]  # 'pi' is the actor network, 'vf' is the critic network
)

model = PPO("MlpPolicy", wrapped_env, policy_kwargs=policy_kwargs, verbose=0, device='cpu')   

vec_env = model.get_env()
obs = vec_env.reset()
#env.render()
time.sleep(1)
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    env.render()
    # VecEnv resets automatically
    # if done:
    #   obs = env.reset()

/home/said/robotics/lib/python3.9/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/said/robotics/lib/python3.9/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
Success ('sword_1', 'r_panda_coll2', -0.0014683094257100515)
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
Success ('sword_1', 'r_panda_coll6', -0.024026262069100385)
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
Success ('sword_1', 'r_panda_coll4', -0.006224055909788528)
-- kin_physx.cpp:ad

In [22]:
from stable_baselines3 import SAC
import time

env = RobotSimEnv(render_mode='human')
model = SAC.load("sac_baranpath",env)
vec_env = model.get_env()
obs = vec_env.reset()
#env.render()
time.sleep(1)
for i in range(200):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    env.render()
    # VecEnv resets automatically
    # if done:
    #   obs = env.reset()

-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]
